In [252]:
#Realizando a previsão com o modelo otimizado
from xgboost import XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np


In [253]:
#Carregando o modelo otimizado
model = XGBClassifier()
model.load_model('../models/xgb_inadimplencia_optuna.json')

In [254]:
#Carregando dados alvo
dados_previsao = pd.read_parquet('../data/intermediate/data_objetivo.parquet', engine='fastparquet')

In [255]:
dados_previsao.head()

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA
0,5058298901476893676,2021-07-01,2021-07-14,2021-08-04,11204.75,4.99
1,274692171162531764,2021-07-01,2021-07-08,2021-08-23,60718.50,5.99
2,274692171162531764,2021-07-01,2021-07-11,2021-08-25,60718.50,5.99
3,274692171162531764,2021-07-01,2021-07-16,2021-08-30,62250.00,5.99
4,465309249432033993,2021-07-01,2021-07-05,2021-07-30,26593.95,6.99


In [256]:
dados_previsao.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA'],
      dtype='object')

In [257]:
dados_previsao.isnull().sum()

ID_CLIENTE                  0
SAFRA_REF                   0
DATA_EMISSAO_DOCUMENTO      0
DATA_VENCIMENTO             0
VALOR_A_PAGAR             131
TAXA                        0
dtype: int64

In [258]:
dados_previsao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12275 entries, 0 to 12274
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CLIENTE              12275 non-null  int64         
 1   SAFRA_REF               12275 non-null  datetime64[ns]
 2   DATA_EMISSAO_DOCUMENTO  12275 non-null  datetime64[ns]
 3   DATA_VENCIMENTO         12275 non-null  datetime64[ns]
 4   VALOR_A_PAGAR           12144 non-null  float64       
 5   TAXA                    12275 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(1)
memory usage: 575.5 KB


In [259]:
dados_previsao.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA'],
      dtype='object')

## Enriquecendo a previsao com informações de cadastro

In [260]:
#Enriquecer a base de dados com dados cadastrais
data_cadastro = pd.read_parquet('../data/intermediate/data_cadastro.parquet', engine='fastparquet')
data_cadastro.columns

Index(['ID_CLIENTE', 'DATA_CADASTRO', 'FLAG_PF', 'SEGMENTO_INDUSTRIAL',
       'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG'],
      dtype='object')

In [261]:
dados_previsao = dados_previsao.merge(data_cadastro, on='ID_CLIENTE', how='left')
dados_previsao.columns


Index(['ID_CLIENTE', 'SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO',
       'VALOR_A_PAGAR', 'TAXA', 'DATA_CADASTRO', 'FLAG_PF',
       'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE', 'CEP_2_DIG'],
      dtype='object')

In [262]:
dados_previsao.isnull().sum()

ID_CLIENTE                  0
SAFRA_REF                   0
DATA_EMISSAO_DOCUMENTO      0
DATA_VENCIMENTO             0
VALOR_A_PAGAR             131
TAXA                        0
DATA_CADASTRO              38
FLAG_PF                    38
SEGMENTO_INDUSTRIAL        38
DOMINIO_EMAIL              38
PORTE                      38
CEP_2_DIG                  38
dtype: int64

## Enriquecendo a base com base nas informações dos clientes e histórico

In [263]:
#Vamos enriquecer a base de dados com as features históricas
dados_historicos = pd.read_parquet('../data/final/data_model_final_with_columns.parquet', engine='fastparquet')
dados_historicos.columns


base_ult_hist = (
    dados_historicos
    .sort_values(['ID_CLIENTE', 'SAFRA_REF'])
    .groupby('ID_CLIENTE', as_index=False)
    .last()
    .drop(columns=[
        'PRAZO_PAGAMENTO_DIAS','DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
        'DATA_VENCIMENTO', 'TAXA', 'DIAS_ATE_PAGAMENTO', 'INADIMPLENTE', 'DIAS_DE_ATRASO', "SAFRA_REF",
        'DATA_CADASTRO', 'FLAG_PF', 'PORTE',
        'MES_EMISSAO', 'REGIAO_CEP_Desconhecido', 'REGIAO_CEP_Nordeste',
        'REGIAO_CEP_Norte', 'REGIAO_CEP_Sudeste', 'REGIAO_CEP_Sul',
        'SEGMENTO_INDUSTRIAL_Indústria', 'SEGMENTO_INDUSTRIAL_PJ',
        'SEGMENTO_INDUSTRIAL_Serviços'
    ])
)

base_ult_hist.columns


Index(['ID_CLIENTE', 'VALOR_A_PAGAR', 'HIST_PAGAMENTOS_ADIANTADOS',
       'HIST_DIAS_ATRASO_ACUMULADO', 'HIST_DIAS_ATRASO_3M',
       'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
       'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO',
       'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS'],
      dtype='object')

## Juntando na base principal

In [264]:
# Faz o join pelo ID_CLIENTE
previsao_enriquecida = dados_previsao.merge(base_ult_hist, on='ID_CLIENTE', how='left', suffixes=('', '_HIST'))

# Substitui VALOR_A_PAGAR ausente pela última disponível
previsao_enriquecida['VALOR_A_PAGAR'] = previsao_enriquecida['VALOR_A_PAGAR'].fillna(previsao_enriquecida['VALOR_A_PAGAR_HIST'])


In [265]:
previsao_enriquecida = (
    previsao_enriquecida
    .assign(
        MES_EMISSAO = previsao_enriquecida['DATA_EMISSAO_DOCUMENTO'].dt.month,
    )
    .assign(
        PRAZO_PAGAMENTO_DIAS = (dados_previsao['DATA_VENCIMENTO'] - dados_previsao['DATA_EMISSAO_DOCUMENTO']).dt.days
    )
    .drop(columns=['DATA_EMISSAO_DOCUMENTO', 'DATA_VENCIMENTO', "DATA_CADASTRO", "DOMINIO_EMAIL"])
)
previsao_enriquecida.isnull().sum()


ID_CLIENTE                      0
SAFRA_REF                       0
VALOR_A_PAGAR                   0
TAXA                            0
FLAG_PF                        38
SEGMENTO_INDUSTRIAL            38
PORTE                          38
CEP_2_DIG                      38
VALOR_A_PAGAR_HIST            228
HIST_PAGAMENTOS_ADIANTADOS    228
HIST_DIAS_ATRASO_ACUMULADO    228
HIST_DIAS_ATRASO_3M           228
HIST_QTD_TRANSACOES           228
HIST_VALOR_MEDIO_PAGO         228
HIST_INADIMPLENTE_ULT_MES     228
HIST_MEDIA_DIAS_ATRASO        228
RENDA_MES_ANTERIOR            228
NO_FUNCIONARIOS               228
MES_EMISSAO                     0
PRAZO_PAGAMENTO_DIAS            0
dtype: int64

### Verificação para nulos

In [224]:
#Conferindo se os nulos estao realmente sendo gerados por clientes novos

colunas_historicas = [
    'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
    'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
    'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO'
]

clientes_possivelmente_novos = previsao_enriquecida[
    previsao_enriquecida[colunas_historicas].isnull().all(axis=1)
]['ID_CLIENTE'].unique()

print(f"Nº de clientes possivelmente novos: {len(clientes_possivelmente_novos)}")


base_cadastro = pd.read_parquet('../data/intermediate/data_cadastro.parquet')

clientes_nao_cadastrados = list(set(clientes_possivelmente_novos) - set(base_cadastro['ID_CLIENTE'].unique()))

print(f"Nº de clientes que NÃO estão nem na base de cadastro: {len(clientes_nao_cadastrados)}")


Nº de clientes possivelmente novos: 88
Nº de clientes que NÃO estão nem na base de cadastro: 21


In [225]:
duplicados = base_cadastro['ID_CLIENTE'].duplicated().sum()
print(f"Nº de duplicados: {duplicados}")


Nº de duplicados: 0


In [227]:
clientes_possivelmente_novos = previsao_enriquecida[
    previsao_enriquecida[colunas_historicas].isnull().all(axis=1)
]['ID_CLIENTE'].unique()

clientes_cadastrados = set(clientes_possivelmente_novos) - set(clientes_nao_cadastrados)

clientes_no_historico = set(dados_historicos['ID_CLIENTE'].unique())

clientes_cadastrados_sem_historico = clientes_cadastrados - clientes_no_historico
clientes_cadastrados_com_historico = clientes_cadastrados & clientes_no_historico

print(f"Clientes cadastrados sem histórico: {len(clientes_cadastrados_sem_historico)}")
print(f"Clientes cadastrados com histórico: {len(clientes_cadastrados_com_historico)}")


#Ou seja, os nulos estão dividos entre clientes cadastrados sem histórico e clientes novos


Clientes cadastrados sem histórico: 67
Clientes cadastrados com histórico: 0


In [268]:
#Preencher históricos com 0

colunas_historicas = [
    'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
    'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
    'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO'
]

# máscara booleana: True para linhas com todos os campos de histórico nulos
mask_sem_historico = previsao_enriquecida[colunas_historicas].isnull().all(axis=1)

previsao_enriquecida.loc[mask_sem_historico, colunas_historicas] = 0
previsao_enriquecida.isnull().sum()


ID_CLIENTE                      0
SAFRA_REF                       0
VALOR_A_PAGAR                   0
TAXA                            0
FLAG_PF                        38
SEGMENTO_INDUSTRIAL            38
PORTE                          38
CEP_2_DIG                      38
VALOR_A_PAGAR_HIST            228
HIST_PAGAMENTOS_ADIANTADOS      0
HIST_DIAS_ATRASO_ACUMULADO      0
HIST_DIAS_ATRASO_3M             0
HIST_QTD_TRANSACOES             0
HIST_VALOR_MEDIO_PAGO           0
HIST_INADIMPLENTE_ULT_MES       0
HIST_MEDIA_DIAS_ATRASO          0
RENDA_MES_ANTERIOR            228
NO_FUNCIONARIOS               228
MES_EMISSAO                     0
PRAZO_PAGAMENTO_DIAS            0
dtype: int64

In [ ]:
#Preencher flag_pf, seg_indsutrial, porte e cep, com base nos valores a pagar 
#(se o valor a pagar é mais proximo da média de uma classe, preencher com essa classe)

# Calcular médias por porte
medias_por_porte = previsao_enriquecida.groupby('PORTE')['VALOR_A_PAGAR'].mean()

# Calcular médias por CEP
medias_por_cep = previsao_enriquecida.groupby('CEP_2_DIG')['VALOR_A_PAGAR'].mean()

# Calcular media por seg_industrial
medias_por_seg_industrial = previsao_enriquecida.groupby('SEGMENTO_INDUSTRIAL')['VALOR_A_PAGAR'].mean()

# Calcular média por flag_pf
medias_por_flag_pf = previsao_enriquecida.groupby('FLAG_PF')['VALOR_A_PAGAR'].mean()

# Função de preenchimento
def preencher_porte(row):
    if pd.notna(row['PORTE']):
        return row['PORTE']
    if pd.isna(row['VALOR_A_PAGAR']):
        return 'Desconhecido'
    return medias_por_porte.sub(row['VALOR_A_PAGAR']).abs().idxmin()

def preencher_cep(row):
    if pd.notna(row['CEP_2_DIG']):
        return row['CEP_2_DIG']
    if pd.isna(row['VALOR_A_PAGAR']):
        return 'Desconhecido'
    return medias_por_cep.sub(row['VALOR_A_PAGAR']).abs().idxmin()


def preencher_seg_industrial(row):
    if pd.notna(row['SEGMENTO_INDUSTRIAL']):
        return row['SEGMENTO_INDUSTRIAL']
    if pd.isna(row['VALOR_A_PAGAR']):
        return 'Desconhecido'
    return medias_por_seg_industrial.sub(row['VALOR_A_PAGAR']).abs().idxmin()

def preencher_flag_pf(row):
    if pd.notna(row['FLAG_PF']):
        return row['FLAG_PF']
    if pd.isna(row['VALOR_A_PAGAR']):
        return 'Desconhecido'
    return medias_por_flag_pf.sub(row['VALOR_A_PAGAR']).abs().idxmin()


previsao_enriquecida['PORTE'] = previsao_enriquecida.apply(preencher_porte, axis=1)
previsao_enriquecida['CEP_2_DIG'] = previsao_enriquecida.apply(preencher_cep, axis=1)
previsao_enriquecida['SEGMENTO_INDUSTRIAL'] = previsao_enriquecida.apply(preencher_seg_industrial, axis=1)
previsao_enriquecida['FLAG_PF'] = previsao_enriquecida.apply(preencher_flag_pf, axis=1)

previsao_enriquecida.isnull().sum()

ID_CLIENTE                      0
SAFRA_REF                       0
VALOR_A_PAGAR                   0
TAXA                            0
FLAG_PF                         0
SEGMENTO_INDUSTRIAL             0
PORTE                           0
CEP_2_DIG                       0
VALOR_A_PAGAR_HIST            228
HIST_PAGAMENTOS_ADIANTADOS      0
HIST_DIAS_ATRASO_ACUMULADO      0
HIST_DIAS_ATRASO_3M             0
HIST_QTD_TRANSACOES             0
HIST_VALOR_MEDIO_PAGO           0
HIST_INADIMPLENTE_ULT_MES       0
HIST_MEDIA_DIAS_ATRASO          0
RENDA_MES_ANTERIOR            228
NO_FUNCIONARIOS               228
MES_EMISSAO                     0
PRAZO_PAGAMENTO_DIAS            0
dtype: int64

In [272]:
#Para preencher as variaveis de renda_mes_anterior, no_funcionarios vamos usar agrupamento por seg e porte

# Agrupar por segmento + porte
medias_renda = previsao_enriquecida.groupby(['SEGMENTO_INDUSTRIAL', 'PORTE'])['RENDA_MES_ANTERIOR'].mean().reset_index()
medias_func = previsao_enriquecida.groupby(['SEGMENTO_INDUSTRIAL', 'PORTE'])['NO_FUNCIONARIOS'].mean().reset_index()

# Merge com a base
previsao_enriquecida = previsao_enriquecida.merge(medias_renda, on=['SEGMENTO_INDUSTRIAL', 'PORTE'], how='left', suffixes=('', '_MEDIA'))
previsao_enriquecida = previsao_enriquecida.merge(medias_func, on=['SEGMENTO_INDUSTRIAL', 'PORTE'], how='left', suffixes=('', '_FUNC'))

# Preencher apenas onde está faltando
previsao_enriquecida['RENDA_MES_ANTERIOR'] = previsao_enriquecida['RENDA_MES_ANTERIOR'].fillna(previsao_enriquecida['RENDA_MES_ANTERIOR_MEDIA'])
previsao_enriquecida['NO_FUNCIONARIOS'] = previsao_enriquecida['NO_FUNCIONARIOS'].fillna(previsao_enriquecida['NO_FUNCIONARIOS_FUNC'])

# Limpar colunas auxiliares
previsao_enriquecida.drop(columns=['RENDA_MES_ANTERIOR_MEDIA', 'NO_FUNCIONARIOS_FUNC'], inplace=True)

previsao_enriquecida.isnull().sum()


ID_CLIENTE                      0
SAFRA_REF                       0
VALOR_A_PAGAR                   0
TAXA                            0
FLAG_PF                         0
SEGMENTO_INDUSTRIAL             0
PORTE                           0
CEP_2_DIG                       0
VALOR_A_PAGAR_HIST            228
HIST_PAGAMENTOS_ADIANTADOS      0
HIST_DIAS_ATRASO_ACUMULADO      0
HIST_DIAS_ATRASO_3M             0
HIST_QTD_TRANSACOES             0
HIST_VALOR_MEDIO_PAGO           0
HIST_INADIMPLENTE_ULT_MES       0
HIST_MEDIA_DIAS_ATRASO          0
RENDA_MES_ANTERIOR             33
NO_FUNCIONARIOS                33
MES_EMISSAO                     0
PRAZO_PAGAMENTO_DIAS            0
dtype: int64

In [273]:
#Para lidarmos com combinações que não aparecem, vamos usar a mediana geral para preencher esses 33 faltantes

# Preencher os valores restantes com a mediana geral
previsao_enriquecida['RENDA_MES_ANTERIOR'] = previsao_enriquecida['RENDA_MES_ANTERIOR'].fillna(dados_historicos['RENDA_MES_ANTERIOR'].median())
previsao_enriquecida['NO_FUNCIONARIOS'] = previsao_enriquecida['NO_FUNCIONARIOS'].fillna(dados_historicos['NO_FUNCIONARIOS'].median())


previsao_enriquecida.isnull().sum()


ID_CLIENTE                      0
SAFRA_REF                       0
VALOR_A_PAGAR                   0
TAXA                            0
FLAG_PF                         0
SEGMENTO_INDUSTRIAL             0
PORTE                           0
CEP_2_DIG                       0
VALOR_A_PAGAR_HIST            228
HIST_PAGAMENTOS_ADIANTADOS      0
HIST_DIAS_ATRASO_ACUMULADO      0
HIST_DIAS_ATRASO_3M             0
HIST_QTD_TRANSACOES             0
HIST_VALOR_MEDIO_PAGO           0
HIST_INADIMPLENTE_ULT_MES       0
HIST_MEDIA_DIAS_ATRASO          0
RENDA_MES_ANTERIOR              0
NO_FUNCIONARIOS                 0
MES_EMISSAO                     0
PRAZO_PAGAMENTO_DIAS            0
dtype: int64

In [282]:
#Por fim, vamos preencher manualmente os nulos que restaram em valor a pagar hist como 0, ja que se tratam
#de clientes sem histórico

previsao_enriquecida = previsao_enriquecida.drop(columns=['VALOR_A_PAGAR_HIST'])
previsao_enriquecida.isnull().sum()


ID_CLIENTE                    0
SAFRA_REF                     0
VALOR_A_PAGAR                 0
TAXA                          0
FLAG_PF                       0
SEGMENTO_INDUSTRIAL           0
PORTE                         0
CEP_2_DIG                     0
HIST_PAGAMENTOS_ADIANTADOS    0
HIST_DIAS_ATRASO_ACUMULADO    0
HIST_DIAS_ATRASO_3M           0
HIST_QTD_TRANSACOES           0
HIST_VALOR_MEDIO_PAGO         0
HIST_INADIMPLENTE_ULT_MES     0
HIST_MEDIA_DIAS_ATRASO        0
RENDA_MES_ANTERIOR            0
NO_FUNCIONARIOS               0
MES_EMISSAO                   0
PRAZO_PAGAMENTO_DIAS          0
dtype: int64

## Com nulos preenchidos adequadamente, vamos diexar as colunas 'CEP_2_DIG' e 'SEGMENTO_INDUSTRIAL' no encoding utilizado no modelo treinado

In [284]:
def map_regiao_cep(cep2):
    try:
        cep2 = int(cep2)
    except:
        return 'Desconhecido'

    if 1 <= cep2 <= 39:
        return 'Sudeste'
    elif cep2 in set(range(40, 50)) | set(range(60, 68)) | {67}:
        return 'Nordeste'
    elif cep2 in {68, 69, 77}:
        return 'Norte'
    elif cep2 in {70, 71, 72, 73, 74, 75, 76}:
        return 'Centro-Oeste'
    elif 80 <= cep2 <= 89:
        return 'Sul'
    else:
        return 'Desconhecido'



previsao_enriquecida['REGIAO_CEP'] = previsao_enriquecida['CEP_2_DIG'].apply(map_regiao_cep)

# Confirme que a coluna foi criada
print(previsao_enriquecida['REGIAO_CEP'].value_counts(dropna=False))


REGIAO_CEP
Sudeste         5235
Sul             2034
Desconhecido    1908
Nordeste        1709
Centro-Oeste     786
Norte            603
Name: count, dtype: int64


In [285]:
# Remover CEP_2_DIG e aplicar one-hot
previsao_enriquecida.drop(columns='CEP_2_DIG', inplace=True)
previsao_enriquecida = pd.get_dummies(previsao_enriquecida, columns=['REGIAO_CEP'], drop_first=True)

In [286]:
# Label Encoding para PORTE (ordinals)
previsao_enriquecida['PORTE'] = previsao_enriquecida['PORTE'].astype('category').cat.codes

In [287]:
# One-Hot Encoding para SEGMENTO_INDUSTRIAL
colunas_onehot = ['SEGMENTO_INDUSTRIAL']
previsao_enriquecida = pd.get_dummies(previsao_enriquecida, columns=colunas_onehot, drop_first=True)

In [299]:
previsao_enriquecida['FLAG_PF'] = (previsao_enriquecida['FLAG_PF'] == 'PF').astype(int)

In [300]:
print("Colunas finais após encoding:")
print(previsao_enriquecida.shape)

Colunas finais após encoding:
(12275, 25)


## Previsão

In [301]:
model.feature_names_in_

array(['VALOR_A_PAGAR', 'TAXA', 'PRAZO_PAGAMENTO_DIAS',
       'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
       'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES',
       'HIST_VALOR_MEDIO_PAGO', 'HIST_INADIMPLENTE_ULT_MES',
       'HIST_MEDIA_DIAS_ATRASO', 'FLAG_PF', 'PORTE', 'RENDA_MES_ANTERIOR',
       'NO_FUNCIONARIOS', 'MES_EMISSAO', 'REGIAO_CEP_Desconhecido',
       'REGIAO_CEP_Nordeste', 'REGIAO_CEP_Norte', 'REGIAO_CEP_Sudeste',
       'REGIAO_CEP_Sul', 'SEGMENTO_INDUSTRIAL_Indústria',
       'SEGMENTO_INDUSTRIAL_PJ', 'SEGMENTO_INDUSTRIAL_Serviços'],
      dtype='<U29')

In [302]:
previsao_enriquecida.columns

Index(['ID_CLIENTE', 'SAFRA_REF', 'VALOR_A_PAGAR', 'TAXA', 'FLAG_PF', 'PORTE',
       'HIST_PAGAMENTOS_ADIANTADOS', 'HIST_DIAS_ATRASO_ACUMULADO',
       'HIST_DIAS_ATRASO_3M', 'HIST_QTD_TRANSACOES', 'HIST_VALOR_MEDIO_PAGO',
       'HIST_INADIMPLENTE_ULT_MES', 'HIST_MEDIA_DIAS_ATRASO',
       'RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS', 'MES_EMISSAO',
       'PRAZO_PAGAMENTO_DIAS', 'REGIAO_CEP_Desconhecido',
       'REGIAO_CEP_Nordeste', 'REGIAO_CEP_Norte', 'REGIAO_CEP_Sudeste',
       'REGIAO_CEP_Sul', 'SEGMENTO_INDUSTRIAL_Indústria',
       'SEGMENTO_INDUSTRIAL_PJ', 'SEGMENTO_INDUSTRIAL_Serviços'],
      dtype='object')

In [303]:
previsao_enriquecida["FLAG_PF"].value_counts()

FLAG_PF
0    12199
1       76
Name: count, dtype: int64

In [304]:
previsao_enriquecida.dtypes

ID_CLIENTE                                int64
SAFRA_REF                        datetime64[ns]
VALOR_A_PAGAR                           float64
TAXA                                    float64
FLAG_PF                                   int64
PORTE                                      int8
HIST_PAGAMENTOS_ADIANTADOS              float64
HIST_DIAS_ATRASO_ACUMULADO              float64
HIST_DIAS_ATRASO_3M                     float64
HIST_QTD_TRANSACOES                     float64
HIST_VALOR_MEDIO_PAGO                   float64
HIST_INADIMPLENTE_ULT_MES               float64
HIST_MEDIA_DIAS_ATRASO                  float64
RENDA_MES_ANTERIOR                      float64
NO_FUNCIONARIOS                         float64
MES_EMISSAO                               int32
PRAZO_PAGAMENTO_DIAS                      int64
REGIAO_CEP_Desconhecido                    bool
REGIAO_CEP_Nordeste                        bool
REGIAO_CEP_Norte                           bool
REGIAO_CEP_Sudeste                      

In [305]:
# Para gerar o entregavel, vamos ter que salvar o ID e SAFRA para juntar apos prever a probabilidade de inadimplencia
# 1. Guardar ID_CLIENTE e SAFRA_REF antes de montar X
ids = previsao_enriquecida[['ID_CLIENTE', 'SAFRA_REF']].copy()

# 2. Selecionar apenas as colunas usadas no modelo
X_previsao = previsao_enriquecida[model.feature_names_in_]

# 3. Gerar previsões de probabilidade (classe 1 = inadimplente)
ids['PROBABILIDADE_INADIMPLENCIA'] = model.predict_proba(X_previsao)[:, 1]


ids.head()



,ID_CLIENTE,SAFRA_REF,PROBABILIDADE_INADIMPLENCIA
0,5058298901476893676,2021-07-01,0.013724
1,274692171162531764,2021-07-01,0.013990
2,274692171162531764,2021-07-01,0.014609
3,274692171162531764,2021-07-01,0.015224
4,465309249432033993,2021-07-01,0.007109


In [310]:
# Salvar
ids.to_csv('../reports/submissao_case.csv', index=False)
ids.to_csv('../submissao_case.csv', index=False)